In [1]:
import numpy as np
import pandas as pd

In [42]:
feature_for_training = np.load('feature_for_training.npy')

In [43]:
feature_for_training_list = []
for element in list(feature_for_training):
    feature_for_training_list.append(int(element))

In [56]:
train = pd.read_csv('E:/kaggle/Benz/data/train.csv/train.csv')

In [45]:
test = pd.read_csv('E:/kaggle/Benz/data/test.csv/test.csv')

In [46]:
feats = np.array(train.columns.values[10:])

In [48]:
feats[feature_for_training_list]

array(['X314', 'X127', 'X261', 'X29', 'X54', 'X276', 'X311'], dtype=object)

In [59]:
train.columns[:10]

Index(['ID', 'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

Use pd.factorize() to encoder the categorical features. One can also use labelencoder to get the same result

In [60]:
train['new_X0'] = pd.factorize(train['X0'])[0]
train['new_X1'] = pd.factorize(train['X1'])[0]
train['new_X2'] = pd.factorize(train['X2'])[0]
train['new_X3'] = pd.factorize(train['X3'])[0]
train['new_X4'] = pd.factorize(train['X4'])[0]
train['new_X5'] = pd.factorize(train['X5'])[0]
train['new_X6'] = pd.factorize(train['X6'])[0]
train['new_X8'] = pd.factorize(train['X8'])[0]

In [63]:
training_features = list(feats[feature_for_training_list]) + ['new_X0', 'new_X1', 'new_X2', 'new_X3', 'new_X4', 'new_X5', 'new_X6', 'new_X8']

In [70]:
X_train = train[training_features]
y_train = train['y']

Use GridSearch to train Random Forest Regressor and XGBoost models

In [66]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

In [78]:
RF_para_grid = {
    'n_estimators' : [100, 200, 300, 400, 500],
    'criterion' : ['mse'],
    'max_depth' : [5,7,9],
    'max_features' : ['auto', 7, 10, 13]
}

In [72]:
import time

In [80]:
RFR = RandomForestRegressor(random_state = 42)
RF_Grid = GridSearchCV(RFR, RF_para_grid, cv = 3, n_jobs = 5, scoring = 'r2')
RF_Grid.fit(X_train, y_train)
RF_model_result = pd.DataFrame(RF_Grid.cv_results_)
RF_model_result.sort_values('rank_test_score', axis = 0, ascending = True, inplace = True)
RF_model_result.to_csv('RF_MODEL_RESULT{}.csv'.format(time.strftime('%Y%m%d%H%M%S', time.localtime())))
joblib.dump(RF_Grid.best_estimator_, 'RF_BEST_MODEL{}.pkl'.format(time.strftime('%Y%m%d%H%M%S', time.localtime())))

['RF_BEST_MODEL20191218210556.pkl']

In [84]:
XGB_para_grid = {
    'learning_rate' : [0.01, 0.05, 0.1],
    'n_estimators' : [100, 200, 300, 400, 500, 700, 800],
    'min_child_weight' : [8,9,11,12],
    'max_depth' : [5,7,9,11],
    'max_features' : ['auto', 7, 10, 13]
}

In [85]:
from xgboost import XGBRegressor

In [86]:
xgb_classifier = XGBRegressor(random_state = 42)
xgb_grid = GridSearchCV(xgb_classifier, XGB_para_grid, cv = 3, n_jobs = 5, scoring = 'r2')
xgb_grid.fit(X_train, y_train)

C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\lenovo\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[22:02:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


AttributeError: module 'pandas' has no attribute 'DataFramea'

In [88]:
xgb_result = pd.DataFrame(xgb_grid.cv_results_)
xgb_result.sort_values('rank_test_score', axis = 0, ascending = True, inplace = True)
xgb_result.to_csv('XGB_MODEL_RESULT{}.csv'.format(time.strftime('%Y%m%d%H%M%S', time.localtime())))
joblib.dump(xgb_grid.best_estimator_, 'XGB_BEST_MODEL{}.pkl'.format(time.strftime('%Y%m%d%H%M%S', time.localtime())))

['XGB_BEST_MODEL20191218220735.pkl']

In [90]:
test['new_X0'] = pd.factorize(test['X0'])[0]
test['new_X1'] = pd.factorize(test['X1'])[0]
test['new_X2'] = pd.factorize(test['X2'])[0]
test['new_X3'] = pd.factorize(test['X3'])[0]
test['new_X4'] = pd.factorize(test['X4'])[0]
test['new_X5'] = pd.factorize(test['X5'])[0]
test['new_X6'] = pd.factorize(test['X6'])[0]
test['new_X8'] = pd.factorize(test['X8'])[0]

In [92]:
prediction = RF_Grid.best_estimator_.predict(test[training_features])

In [94]:
prediction

array([ 80.79769833,  97.09104226,  79.36496502, ...,  94.40245884,
       110.39337287,  93.80873807])

In [96]:
test['y'] = pd.Series(prediction)

In [98]:
test[['ID','y']].to_csv('my_submission1.csv')